In [ ]:
import sys
import matplotlib.pyplot as plt
import torch

torch.backends.cudnn.benchmark = True
import numpy as np

torch.set_float32_matmul_precision('medium')
import h5py
from tqdm.auto import tqdm
import dac
from audiotools import AudioSignal
import torchaudio
from torch import nn, optim
import torch.nn.functional as F
import torch
from torch.utils.data import Dataset, DataLoader


class Encoder(nn.Module):
    def __init__(self, in_channels, hidden_channels, n_embeddings, embedding_dim):
        super(Encoder, self).__init__()
        self.dropout = nn.Dropout(p=0.2)
        self.conv1 = nn.Conv2d(in_channels, hidden_channels, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.Conv2d(hidden_channels, hidden_channels, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.Conv2d(hidden_channels, embedding_dim, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        # x = self.dropout(F.relu(self.conv1(x)))
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.conv3(x)
        return x


class Decoder(nn.Module):
    def __init__(self, embedding_dim, hidden_channels, out_channels):
        super(Decoder, self).__init__()
        self.dropout = nn.Dropout(p=0.2)
        self.conv1 = nn.ConvTranspose2d(embedding_dim, hidden_channels, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.ConvTranspose2d(hidden_channels, hidden_channels, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.ConvTranspose2d(hidden_channels, out_channels, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        # x = self.dropout(F.relu(self.conv1(x)))
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = torch.sigmoid(self.conv3(x))
        return x


class VectorQuantizer(nn.Module):
    def __init__(self, n_embeddings, embedding_dim, commitment_cost=0.25):
        super(VectorQuantizer, self).__init__()
        self.embedding_dim = embedding_dim
        self.n_embeddings = n_embeddings
        self.embeddings = nn.Embedding(n_embeddings, embedding_dim)
        self.embeddings.weight.data.uniform_(-1 / n_embeddings, 1 / n_embeddings)
        self.commitment_cost = commitment_cost

    def forward(self, inputs):
        flat_input = inputs.view(-1, self.embedding_dim)
        distances = (torch.sum(flat_input ** 2, dim=1, keepdim=True)
                     + torch.sum(self.embeddings.weight ** 2, dim=1)
                     - 2 * torch.matmul(flat_input, self.embeddings.weight.t()))

        encoding_indices = torch.argmin(distances, dim=1).unsqueeze(1)
        quantized = torch.index_select(self.embeddings.weight, 0, encoding_indices.view(-1))
        quantized = quantized.view(inputs.shape)

        # Commitment Loss
        e_latent_loss = F.mse_loss(quantized.detach(), inputs)
        q_latent_loss = F.mse_loss(quantized, inputs.detach())
        loss = q_latent_loss + self.commitment_cost * e_latent_loss

        quantized = inputs + (quantized - inputs).detach()
        return quantized, loss, encoding_indices


class VQVAE2(nn.Module):
    def __init__(self, in_channels=1, hidden_channels=64, n_embeddings=512, embedding_dim=64):
        super(VQVAE2, self).__init__()
        self.encoder = Encoder(in_channels, hidden_channels, n_embeddings, embedding_dim)
        self.vq = VectorQuantizer(n_embeddings, embedding_dim)
        self.decoder = Decoder(embedding_dim, hidden_channels, in_channels)

    def forward(self, x):
        z = self.encoder(x)
        quantized, vq_loss, _ = self.vq(z)
        x_recon = self.decoder(quantized)
        x_recon = F.interpolate(x_recon, size=x.shape[-2:], mode='bilinear', align_corners=False)
        return x_recon, vq_loss


class SnippetDatasetHDF(Dataset):
    def __init__(self, hdf, scaling='minmax'):
        self.num_rows = 0
        self.size = int(3.4 * 24000)  # fixed size for samples
        self.scaling = scaling
        self.data = self.create_data(hdf)

        if scaling == 'standard':
            self.mean = self.data.mean()
            self.std = self.data.std()
            self.data = (self.data - self.mean) / self.std

        elif scaling == 'minmax':
            self.min = self.data.min()
            self.max = self.data.max()
            self.data = (self.data - self.min) / (self.max - self.min)

    def __len__(self):
        return self.num_rows

    def __getitem__(self, idx):
        return self.data[idx]

    def create_data(self, hdf):
        data = []
        keys = list(hdf.keys())
        self.num_rows = len(keys)
        for key in tqdm(keys):
            sample = hdf[key]['audio'][:]
            if len(sample) > self.size:
                self.num_rows -= 1
                continue

            if len(sample) < self.size:
                sample = np.pad(sample, (0, self.size - len(sample)), 'constant')

            data.append(sample)

        return torch.tensor(np.array(data)).float()

    def retransform(self, data):
        if self.scaling == 'standard':
            return data * self.std + self.mean
        elif self.scaling == 'minmax':
            return data * (self.max - self.min) + self.min


def train_vqvae2(model, dataloader, epochs=15, lr=1e-5):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    #criterion = nn.MSELoss()
    criterion = nn.L1Loss()
    for epoch in range(epochs):
        running_loss = 0.0
        for latents in dataloader:
            optimizer.zero_grad()
            outputs, vq_loss = model(latents)
            loss = criterion(outputs, latents) + vq_loss
            loss.backward(retain_graph=True)
            # nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(dataloader)}')


class LatentDataset(Dataset):
    def __init__(self, latents_tensor):
        self.latents_tensor = latents_tensor

    def __len__(self):
        return len(self.latents_tensor)

    def __getitem__(self, idx):
        return self.latents_tensor[idx]


def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
            nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)


loaded_data = torch.load('latent_dataset.pt', weights_only=False)
latent_dataset = LatentDataset(loaded_data)
train_loader = DataLoader(latent_dataset, batch_size=16, shuffle=True)

vqvae2_model = VQVAE2()
initialize_weights(vqvae2_model)
train_vqvae2(vqvae2_model, train_loader)


def generate_samples(model, num_samples=1, save_path='generated_sample.wav', sampling_rate=24000, output_length=24000):
    model.eval()
    with torch.no_grad():
        dummy_input = torch.randn(1, 1, 8, 4096).to(next(model.parameters()).device)
        z = model.encoder(dummy_input)
        _, _, latent_height, latent_width = z.shape

        encoding_indices = torch.randint(0, model.vq.n_embeddings, (num_samples, latent_height, latent_width)).to(
            next(model.parameters()).device)
        quantized_latents = model.vq.embeddings(encoding_indices).view(num_samples, model.vq.embedding_dim,
                                                                       latent_height, latent_width)

        generated_samples = model.decoder(quantized_latents)
        print("Generated samples shape:", generated_samples.size())

        generated_samples = generated_samples.view(num_samples, -1)
        generated_waveform = F.interpolate(generated_samples.unsqueeze(1), size=(output_length,), mode='linear',
                                           align_corners=False)
        generated_waveform = generated_waveform.squeeze().cpu()
        generated_waveform = generated_waveform / torch.max(torch.abs(generated_waveform))
        generated_waveform_np = generated_waveform.numpy().astype(np.float32)
        torchaudio.save(save_path, torch.tensor(generated_waveform_np).unsqueeze(0), sampling_rate)

        print(f"Sample saved to {save_path}")

        plt.figure(figsize=(10, 4))
        plt.plot(generated_waveform_np)
        plt.title('Generated Waveform')
        plt.xlabel('Sample Index')
        plt.ylabel('Amplitude')
        plt.show()

        return generated_waveform_np


generate_samples(vqvae2_model, output_length=48000)



